# BERTopic news processor

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime

plt.rcParams['figure.figsize'] = [15, 7.5]
plt.rcParams['figure.facecolor'] = 'white'
#plt.style.use('dark_background')

comma_strip = lambda x: x.rstrip(",")

import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords \
              and token != " " \
              and token.strip() not in punctuation
              and token.strip() not in ["«", "»", "“", "”"]]
    text = " ".join(tokens)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dormant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
try:
    news = pd.read_csv('../DATA-MINING/SCRAPED-DATA/LEMM-NEWS-2018-1-1-2022-9-1.csv',
                       sep='\t',
                       on_bad_lines='skip')
except:
    news = pd.read_csv('../DATA-MINING/SCRAPED-DATA/NEWS-2018-1-1-2022-9-1.csv',
                       sep='\t',
                       on_bad_lines='skip')
    news['title'] = news['title'].apply(preprocess_text)
    news.to_csv('../DATA-MINING/SCRAPED-DATA/LEMM-NEWS-2018-1-1-2022-9-1.csv',
                sep='\t',
                index=False)
    
    # validate working file
    news = pd.read_csv('../DATA-MINING/SCRAPED-DATA/LEMM-NEWS-2018-1-1-2022-9-1.csv',
                       sep='\t',
                       on_bad_lines='skip')

# DEBUG

In [3]:
#news = news.iloc[:40000]

In [4]:
print(f'Number of news headers: {len(news)}')
news.head()

Number of news headers: 411713


,date,title
0,2018-01-01,потанин оставаться вода
1,2018-01-01,автор хит 1 1 устраивать праздничный переполох
2,2018-01-01,северный корея принимать участие олимпиада южный
3,2018-01-01,зимний сказка петербург продлиться недолго
4,2018-01-01,алкогольный энергетик показывать красный свет


In [5]:
datify = lambda x: datetime.strptime(x, '%Y-%m-%d')
textify = lambda x: x.strftime('%Y-%m-%d')

start_d, end_d = datify(min(news['date'])), datify(max(news['date']))
delta_d = end_d - start_d
num_days = delta_d.days

print(f"From {textify(start_d)} to {textify(end_d)}")
print(f"Number of days: {num_days}")

num_bins = int(num_days/7)
print(f"{num_bins} is going to be the number of weekly bins.")

From 2018-01-01 to 2022-09-01
Number of days: 1704
243 is going to be the number of weekly bins.


In [6]:
news.tail()

,date,title
411708,2022-09-01,петербуржец напоминать изменяться общественный...
411709,2022-09-01,обгорать трешка проспект наука вытаскивать пос...
411710,2022-09-01,называть креативный двор санкт-петербург котор...
411711,2022-09-01,первый сентябрь петербург открываться 7 социал...
411712,2022-09-01,гороскоп весь знак зодиак 1 сентябрь 2022 год ...


In [ ]:
from bertopic import BERTopic
from dateutil.relativedelta import relativedelta

# Topics to visualize (-2 to include all)
num_of_topics = -2

# Number of topics after reduction
reduce_topics_to = 50

# Start topic
start_topic = 2

timestamps = news['date'].to_list()
titles = news['title'].to_list()

try:
    topic_df = pd.read_csv('./PROCESSED-DATA/TOPICS.csv', sep=',', on_bad_lines='skip')
    topic_df.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
    topic_df.set_index('Date', inplace=True)
    topic_df.index = topic_df.index.map(datify)
    topic_df = topic_df.iloc[:,start_topic:num_of_topics+1]
except:
    topic_model = BERTopic(embedding_model='distiluse-base-multilingual-cased-v1',
                           verbose=True)
    topics, probs = topic_model.fit_transform(titles)
    topic_model.reduce_topics(titles, nr_topics=reduce_topics_to)
    topic_labels = topic_model.generate_topic_labels(nr_words=3,
                                                     topic_prefix=True,
                                                     word_length=10,
                                                     separator="_")
    
    topics_over_time = topic_model.topics_over_time(titles, timestamps, nr_bins=num_bins)
    topic_df = pd.DataFrame(topics_over_time)
    topic_df.rename(columns={"Timestamp": "Date"}, inplace=True)
    topic_df.fillna(0.0, inplace=True)
    
    new_index = pd.date_range(start=textify(start_d),
                              end=textify(end_d + relativedelta(months=-1)),
                              freq='MS')
    
    topic_df = topic_df.pivot(index="Date",
                              columns="Topic",
                              values="Frequency")
    
    topic_df = topic_df.reindex(topic_df.index.union(new_index)).interpolate(method='time')
    topic_df = topic_df.reindex(new_index)
    topic_df.set_axis(topic_labels, axis=1, inplace=True)
    topic_df.fillna(0.0, inplace=True)
    
    topic_df.to_csv('./PROCESSED-DATA/TOPICS.csv', sep=',')
    
    # validate working file
    topic_df = pd.read_csv('./PROCESSED-DATA/TOPICS.csv', sep=',', on_bad_lines='skip')
    topic_df.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
    topic_df.set_index('Date', inplace=True)
    topic_df.index = topic_df.index.map(datify)
    topic_df = topic_df.iloc[:,start_topic:num_of_topics+1]
    
print(f"Number of topics: {len(topic_df.columns)}")
plt.plot(topic_df);
if len(topic_df.columns) <= 35:
    plt.legend(topic_df.columns);

2023-06-06 17:52:08.727874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 17:52:09.074406: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-06 17:52:09.074431: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-06 17:52:09.131681: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-06 17:52:10.146168: W tensorflow/stream_executor/platform/de

Batches:   0%|          | 0/12867 [00:00<?, ?it/s]

2023-06-06 19:33:14,281 - BERTopic - Transformed documents to Embeddings


In [ ]:
topic_df.head()

## Take only TS with peaks

In [ ]:
import statsmodels.tsa.api as smt

# Desired number of topics
desired_topics = 20

for threshold in np.arange(10.0, 0.1, -0.1):
    non_stat_topic_df = topic_df.copy()
    for col in non_stat_topic_df.columns:
        if non_stat_topic_df[col].max() / non_stat_topic_df[col].mean() < threshold:
            non_stat_topic_df = non_stat_topic_df.drop(columns=[col])
    # If num of topics is desired += 10%
    if np.abs(len(non_stat_topic_df.columns) - desired_topics) <= (desired_topics / 10):
        break

plt.plot(non_stat_topic_df);
plt.legend(non_stat_topic_df.columns);